<a href="https://colab.research.google.com/github/Nittaany/Lingua-Box/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gradio gtts PyPDF2 deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
!pip install pdfplumber pytesseract Pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 45.0 MB/s eta 0:00:00


In [ ]:
import os
print([f for f in os.listdir() if 'torch' in f])


[]


In [ ]:
from transformers import pipeline
from gtts import gTTS
from IPython.display import Audio
import tempfile
import PyPDF2
import pdfplumber
import pytesseract
from deep_translator import GoogleTranslator
import gradio as gr
from PIL import Image

# Load summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# PDF Text Extractor with OCR support for scanned PDFs
def extract_text_from_pdf(file):
    text = ""
    try:
        with pdfplumber.open(file) as pdf:
            for page in pdf.pages:
                extracted = page.extract_text()
                if not extracted:  # If no text is extracted, use OCR
                    images = page.images
                    for img in images:
                        image = Image.open(img['src'])
                        extracted = pytesseract.image_to_string(image)
                if extracted:
                    text += extracted
    except Exception as e:
        print("Error extracting PDF text:", e)
    return text

# Translation using deep-translator
def translate_text(text, target_lang):
    try:
        translated = GoogleTranslator(source='auto', target=target_lang).translate(text)
        return translated
    except Exception as e:
        print("Translation error:", e)
        return text  # fallback to English if translation fails

# TTS function
def text_to_speech(text, lang):
    try:
        tts = gTTS(text, lang=lang)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            tts.save(fp.name)
            return fp.name
    except Exception as e:
        print("TTS error:", e)
        return None

# Clean extracted text
def clean_text(text):
    # Remove extra spaces, line breaks, special characters, etc.
    text = text.replace("\n", " ").strip()
    text = ' '.join(text.split())  # Remove extra spaces
    return text

# Main processing function
def summarize_translate_speak(input_text, target_language, file):
    # If file is uploaded, extract text
    if file is not None:
        input_text = extract_text_from_pdf(file)

    # Clean the extracted text
    input_text = clean_text(input_text)

    # Check if input is valid
    if not input_text or input_text.strip() == "":
        return "No valid text provided for summarization.", None

    # Truncate if too long (BART limit is ~1024 tokens)
    if len(input_text) > 2000:
        input_text = input_text[:2000]

    # Summarize
    result = summarizer(input_text, max_length=100, min_length=30, do_sample=False)
    summary = result[0]['summary_text']

    # Language codes for gTTS
    language_codes = {
        "English": "en",
        "Hindi": "hi",
        "Tamil": "ta",
        "Malayalam": "ml",
        "Bengali": "bn",
        "Telugu": "te"
    }

    lang_code = language_codes.get(target_language, "en")

    # Translate
    translated_summary = translate_text(summary, lang_code)

    # TTS
    audio_path = text_to_speech(translated_summary, lang_code)

    # Debug logs (optional)
    print("Summary:", summary)
    print("Translated:", translated_summary)
    print("Audio path:", audio_path)

    return translated_summary, audio_path

# Gradio Interface
with gr.Blocks(css="""
body {
    background: linear-gradient(135deg, #1a1a2e, #16213e, #0f3460);
    color: #f1f1f1;
    font-family: 'Segoe UI', sans-serif;
    padding: 0;
    margin: 0;
}
#title {
    font-size: 2.5em;
    font-weight: bold;
    color: #4ade80;
    text-align: center;
    margin-top: 30px;
    text-shadow: 0 0 8px #4ade80;
}
#description {
    text-align: center;
    font-size: 1.2em;
    margin-bottom: 30px;
    color: #a1a1aa;
}
.section {
    background-color: rgba(30, 30, 30, 0.9);
    padding: 30px;
    border-radius: 15px;
    box-shadow: 0 0 20px rgba(0,0,0,0.5);
    border: 1px solid #2e2e2e;
}
.gr-button {
    background: linear-gradient(to right, #14b8a6, #4ade80);
    color: black;
    font-weight: bold;
    border: none;
    border-radius: 10px;
    padding: 15px 20px;
    font-size: 1.1em;
    box-shadow: 0 0 10px #4ade80;
    transition: all 0.3s ease;
}
.gr-button:hover {
    background: linear-gradient(to right, #4ade80, #14b8a6);
    box-shadow: 0 0 20px #14b8a6;
}
.gr-textbox input {
    background-color: #1e1e1e;
    border: 1px solid #333;
    color: #fff;
    font-size: 1.1em;
    padding: 12px;
    border-radius: 8px;
}
.gr-textbox input:focus {
    border-color: #4ade80;
}
.gr-dropdown select {
    background-color: #1e1e1e;
    color: #fff;
    border: 1px solid #333;
    font-size: 1.1em;
    padding: 12px;
    border-radius: 8px;
}
.gr-dropdown select:focus {
    border-color: #4ade80;
}
""") as interface:
    gr.Markdown("<div id='title'>Lingua Box 🎁</div>")
    gr.Markdown("<div id='description'>A box of language tools — summary, speech, and translation</div>")

    with gr.Row():
        with gr.Column(scale=1, elem_classes="section"):
            gr.Markdown("### 🧾 Input Text or PDF Report")
            input_text = gr.Textbox(lines=10, placeholder="Paste your text here...", label="📝 Report Text")
            file_input = gr.File(label="📄 Or Upload PDF Report")
            lang_dropdown = gr.Dropdown(
                ["English", "Hindi", "Tamil", "Malayalam", "Bengali", "Telugu"],
                label="Select Output Language 🌐",
                value="English"
            )
            submit_btn = gr.Button("🚀 Summarize & Translate")

        with gr.Column(scale=1, elem_classes="section"):
            gr.Markdown("### ✨ Output Summary & Audio")
            output_text = gr.Textbox(label="📝 Simplified Summary")
            output_audio = gr.Audio(label="🔊 Summary Voice", type="filepath")

    submit_btn.click(fn=summarize_translate_speak,
                     inputs=[input_text, lang_dropdown, file_input],
                     outputs=[output_text, output_audio])

interface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b9f30cf583b590681.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
